In [1]:
import pandas as pd
import pickle
import datetime

In [2]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
current_year = date.strftime("%Y")
print(f"Current Year -> {current_year}")

Current Year -> 2025


In [3]:
previous_year = str(pd.to_numeric(current_year) - 1)
print(f"Previous Year -> {previous_year}")

Previous Year -> 2024


In [4]:
#Import Files

#Data used as basis
Solar_Data = "../Excel Resources/pvwatts_hourly.xlsx"

#Bitcoin Data
hashrate_BTC = "Input Files/hash-rate.json"
revenue_BTC = "Input Files/miners-revenue.json"
marketprice_BTC = "Input Files/market-price.json"


In [5]:
#Variables

#BTC Mining
Basis_MW = 100 #MW
Hardware_Eff = 0.88 #Percentage
W_from_Renewables = 507 #GW
P_from_Renewables = 0.3 #GW
P_from_Solar = 0.1 
Solar_Plant_W = 100 #MW
Cooling = 0.2 #Percentage enery needed for cooling and other operational overhead energy consumption

#Solar Plant
PV_plant_capacity  = 500 #MW
AC_capacity = 100 #MW PV watts data plant capacity used when extracting data from the website
Solar_PV_Cost = 4400000 #$/MW
Capital_Cost_PV = Solar_PV_Cost*PV_plant_capacity #$/kW
PV_Plant_Cost = PV_plant_capacity * Capital_Cost_PV #$
Off_peak_hours = [22,23,0,1,2,3,4,5]

#ESKOM Solar Plant Requested Demand
Energy_Demand = 150 #MW
ZAR_To_USD = 1/18.8963
Electricity_Cost = 1.0398 #ZAR per kWh for sale to the grid
Electricity_Cost_USD = Electricity_Cost*ZAR_To_USD #USD per kWh for households

#Calculations based on Assumptions
W_from_Solar = W_from_Renewables*P_from_Solar
Global_W = W_from_Renewables/P_from_Renewables

#Months
months = range(1, 13)
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
hours = range(24)

#Date of data extraction from Blockchain.com
last_date = '25-October-2024' #date of data export
year = '2023' #previous year to current

Data retrieved from 

In [6]:
data = {
    'Model': [
        'MicroBT WhatsMiner M63S', 'Bitmain Antminer S21 Hyd (335Th)', 'MicroBT WhatsMiner M63',
        'MicroBT WhatsMiner M66S', 'Bitmain Antminer S21 Pro (234Th)', 'Bitmain Antminer S21 (200Th)',
        'Bitmain Antminer T21 (190Th)', 'MicroBT WhatsMiner M60S', 'Canaan Avalon A1566',
        'MicroBT WhatsMiner M60', 'Bitmain Antminer S19j Pro (104Th)', 'Bitmain Antminer S19 Pro (110Th)',
        'Core i7 950', 'Atom N450', 'ATI 4850', 'ATI 5770', 'The Single', 'Rig Box'
    ],
    'Release': [
        '01-Nov-23', '01-Feb-24', '01-Nov-23', '01-Nov-23', '01-Jul-24', '01-Feb-24', 
        '01-Feb-24', '01-Feb-24', '01-Oct-24', '01-Feb-24', '01-Jul-21', '01-May-20', 
        '2009', '2009', '2010', '2010', '2011', '2011'
    ],
    'Hashrate (Th/s)': [
        390.0, 335.0, 334.0, 298.0, 234.0, 200.0, 190.0, 186.0, 185.0, 172.0, 
        104.0, 110.0, 0.000019, 0.0001, 0.000101, 0.000215, 0.000832, 0.05
    ],
    'Power (W)': [
        7215, 5360, 6646, 5513, 3531, 3550, 3610, 3441, 3420, 3422, 
        3068, 3250, 157.5, 322.58, 109.78, 110, 80, 2450.98
    ],
    'Algorithm': [
        'SHA-256'] * 18,
    'Income ($/day)': [
        20.29, 17.43, 17.38, 15.51, 12.18, 10.41, 9.89, 9.68, 9.63, 8.95, 
        4.58, 4.84, None, None, None, None, None, None
    ],
    'Electricity Cost ($/day)': [
        -20.78, -15.44, -19.14, -15.88, -10.17, -10.08, -10.40, -9.91, -9.85, -9.86, 
        -8.84, -9.36, None, None, None, None, None, None
    ],
    'With Electricity': [
        -0.49, 1.99, -1.76, -0.37, 2.01, 0.33, -0.51, -0.23, -0.22, -0.91, 
        -4.26, -4.52, None, None, None, None, None, None
    ],
    'Without Electricity': [
        20.29, 17.43, 17.38, 15.51, 12.18, 10.41, 9.89, 9.68, 9.63, 8.95, 
        4.58, 4.84, None, None, None, None, None, None
    ],
    'Efficiency (j/Gh)': [
        0.019, 0.016, 0.020, 0.019, 0.015, 0.017, 0.019, 0.019, 0.018, 0.020, 
        0.030, 0.030, None, None, None, None, None, None
    ],
    'Price ($)': [
        12449, 5889, 6879, 9649, 5029, 2949, 3159, 3829, 5359.67, 3329, 
        770, 812, 70, 15, 90, 30, 795, 90
    ],
    'Noise db': [
        50, 50, 50, 50, 75, 75, 75, 75, 75, 75, 
        75, 75, None, None, None, None, None, None
    ],
    'Cooling': [
        'hydrocooling', 'hydrocooling', 'hydrocooling', 'hydrocooling', '2 fans', '2 fans', 
        '2 fans', '2 fans', '2 fans', '2 fans', 
        '4 fans', '4 fans', None, None, None, None, None, None
    ],
    'Seller': [
        'AsicMarketPlace', 'AsicMarketPlace', 'AsicMarketPlace', 'AsicMarketPlace', 
        'AsicMarketPlace', 'AsicMarketPlace', 'AsicMarketPlace', 'AsicMarketPlace', 
        'MillionMiner', 'AsicMarketPlace', 'Apexto Mining', 'Scminer', None, None, None, None, None, None
    ],
    'Days to Breakeven': [
        613.553475, 337.865749, 395.799770, 622.114765, 412.889984, 283.285303, 
        319.413549, 395.557851, 556.559709, 371.955307, 168.122271, 167.768595, None, None, None, None, None, None
    ],
    'Rig Type': [
        'ASIC', 'ASIC', 'ASIC', 'ASIC', 'ASIC', 'ASIC', 'ASIC', 'ASIC', 
        'ASIC', 'ASIC', 'ASIC', 'ASIC', 'CPU', 'CPU', 'GPU', 'GPU', 'FPGA', 'FPGA'
    ]
}



In [7]:
Crypto_Data = pd.DataFrame(data)
Crypto_Data.head()

,Model,Release,Hashrate (Th/s),Power (W),Algorithm,Income ($/day),Electricity Cost ($/day),With Electricity,Without Electricity,Efficiency (j/Gh),Price ($),Noise db,Cooling,Seller,Days to Breakeven,Rig Type
0,MicroBT WhatsMiner M63S,01-Nov-23,390.0,7215.0,SHA-256,20.29,-20.78,-0.49,20.29,0.019,12449.0,50.0,hydrocooling,AsicMarketPlace,613.553475,ASIC
1,Bitmain Antminer S21 Hyd (335Th),01-Feb-24,335.0,5360.0,SHA-256,17.43,-15.44,1.99,17.43,0.016,5889.0,50.0,hydrocooling,AsicMarketPlace,337.865749,ASIC
2,MicroBT WhatsMiner M63,01-Nov-23,334.0,6646.0,SHA-256,17.38,-19.14,-1.76,17.38,0.020,6879.0,50.0,hydrocooling,AsicMarketPlace,395.799770,ASIC
3,MicroBT WhatsMiner M66S,01-Nov-23,298.0,5513.0,SHA-256,15.51,-15.88,-0.37,15.51,0.019,9649.0,50.0,hydrocooling,AsicMarketPlace,622.114765,ASIC
4,Bitmain Antminer S21 Pro (234Th),01-Jul-24,234.0,3531.0,SHA-256,12.18,-10.17,2.01,12.18,0.015,5029.0,75.0,2 fans,AsicMarketPlace,412.889984,ASIC


In [8]:
#Save variables

with open('Global_Variables.pkl', 'wb') as f:
    pickle.dump({'Crypto_Data' : Crypto_Data,
                 
                 #Mining Assumptions
                'Basis_MW': Basis_MW,
                'Hardware_Eff': Hardware_Eff,
                'W_from_Renewables': W_from_Renewables,
                'P_from_Renewables': P_from_Renewables,
                'P_from_Solar': P_from_Solar,
                'Cooling': Cooling,
                'Solar_Plant_W': Solar_Plant_W,

                #Solar Plant
                #'PV_Capacity': PV_Capacity,
                'AC_capacity' : 100, #MW Maximum capacity
                'PV_plant_capacity'  : 500, #MW
                'Capital_Cost_PV': Capital_Cost_PV,
                'PV_Plant_Cost': PV_Plant_Cost,
                'Electricity_Cost_USD' : Electricity_Cost_USD,

                #Calculations based on Assumptions
                'W_from_Solar': W_from_Solar,
                'Global_W': Global_W,
                'Off_peak_hours' : Off_peak_hours,
                'Energy_Demand' : Energy_Demand,

                #Data paths
                'Solar_Data' : Solar_Data,
                'hashrate_BTC' : hashrate_BTC,
                'revenue_BTC' : revenue_BTC,
                'marketprice_BTC' : marketprice_BTC,

                'current_year' : current_year,
                'previous_year' : previous_year,
                'last_date' : last_date,
                'months' : months,
                'days_in_month' : days_in_month,
                'hours' : hours,
                 }, f)